# Libraries

In [1]:
import os
import pandas as pd
import swifter

In [2]:
print(os.getcwd())

/Users/bhaskarn/Downloads/VA_Data/src


# Custom functions 

In [3]:
# get name of dataframe (which is required)
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

# get shape of dataframe
def print_shape(df):
    df_name = get_df_name(df)
    print(f'shape of the df({df_name}):', df.shape)

# Data sourcing

#### Read all the csv files

In [4]:
data_location = '~/Downloads/VA_Data/Datasets/'
train_raw = pd.read_csv(data_location + 'Train.csv')
camp_raw = pd.read_csv(data_location + 'Health_Camp_Detail.csv')
patient_raw = pd.read_csv(data_location + 'Patient_Profile.csv')
first_camp = pd.read_csv(data_location + 'First_Health_Camp_Attended.csv')
second_camp = pd.read_csv(data_location + 'Second_Health_Camp_Attended.csv')
third_camp = pd.read_csv(data_location + 'Third_Health_Camp_Attended.csv')
test_raw = pd.read_csv(data_location + 'test_l0Auv8Q.csv')

In [5]:
print_shape(train_raw)
print_shape(camp_raw)
print_shape(patient_raw)
print_shape(first_camp)
print_shape(second_camp)
print_shape(third_camp)
print_shape(test_raw)

shape of the df(train_raw): (75278, 8)
shape of the df(camp_raw): (65, 6)
shape of the df(patient_raw): (37633, 11)
shape of the df(first_camp): (6218, 5)
shape of the df(second_camp): (7819, 3)
shape of the df(third_camp): (6515, 4)
shape of the df(test_raw): (35249, 8)


# Understand the data

#### Understand Train.csv

In [6]:
train_raw.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [7]:
# check for missing values
train_raw.isna().sum()

Patient_ID             0
Health_Camp_ID         0
Registration_Date    334
Var1                   0
Var2                   0
Var3                   0
Var4                   0
Var5                   0
dtype: int64

In [8]:
# we have missing vales in Registration_Date
# let's check if the patient_ids are unique
train_raw.Patient_ID.nunique()

29828

In [9]:
# no of unique patient_ids are different form no. of rows in train_raw
# let's investigate little deeper
train_raw.Patient_ID.value_counts()

516956    32
490196    28
513633    28
509188    28
517006    25
          ..
507996     1
512094     1
514143     1
524388     1
524288     1
Name: Patient_ID, Length: 29828, dtype: int64

In [10]:
# we have duplicate patient_ids, now it's clear why no of unique patient_ids are less
# let's try to understand why we have duplicate patient_ids
patient_516956_data = train_raw.loc[train_raw['Patient_ID'] == 516956]
patient_516956_data

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
34,516956,6541,19-Dec-05,7,4,0,0,5
389,516956,6587,03-Aug-05,6,4,0,0,5
5002,516956,6540,31-Oct-04,0,0,0,0,0
7279,516956,6539,06-Aug-04,0,0,0,0,0
8557,516956,6535,03-Dec-03,0,0,0,0,0
9211,516956,6554,08-May-05,6,4,0,0,5
9260,516956,6581,02-Jan-04,0,0,0,0,0
12151,516956,6543,11-Nov-06,7,4,0,0,5
16488,516956,6578,20-Aug-05,6,4,0,0,5
20738,516956,6580,19-Oct-04,4,1,0,0,4


In [11]:
# it is possible few patients attend camp regularly
# let's see if all the instaces are unique (check by date)
print(patient_516956_data.Registration_Date.nunique()==patient_516956_data.shape[0])

False


In [12]:
# looks like we have duplicate registration dates as well
# lets check the shape of this particular patients data
patient_516956_data.shape

(32, 8)

In [13]:
# now lets check how many unique visits he has paid
patient_516956_data.Registration_Date.nunique()

30

In [14]:
# alright, registration dates are not matching shape of the patient data
# lets check for this patient's duplicate registration dates
pd.concat(g for _, g in patient_516956_data.groupby('Registration_Date') if len(g) > 1)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
33173,516956,6532,24-Feb-05,4,1,0,0,4
42618,516956,6523,24-Feb-05,4,1,0,0,4
5002,516956,6540,31-Oct-04,0,0,0,0,0
40731,516956,6526,31-Oct-04,4,1,0,0,4


In [15]:
# therefore, there could be multiple registrations by same individual
# let's check the attendance in each camp

camp_dfs = [first_camp, second_camp, third_camp]
for df in camp_dfs:
    try:
        pd.concat(i for _, i in df.groupby(['Patient_ID', 'Health_Camp_ID']) if len(i) > 1)
    except ValueError as v:
        print(str(v), f'in {get_df_name(df)}')
    
# it is evident that there are no patients that have gone to camp twice
# which is what we wanted

No objects to concatenate in first_camp
No objects to concatenate in second_camp
No objects to concatenate in third_camp


# Data preparation

In [16]:
train_raw.describe()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5
count,75278.000000,75278.000000,75278.000000,75278.000000,75278.000000,75278.000000,75278.000000
mean,507233.325660,6547.597319,0.796156,0.238556,0.000385,0.022424,0.248386
std,12404.734886,19.272983,7.626687,3.769594,0.019624,0.199570,1.213938
min,485679.000000,6523.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,496505.250000,6534.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,507276.500000,6541.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,517919.000000,6562.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,528657.000000,6587.000000,288.000000,156.000000,1.000000,4.000000,31.000000


In [17]:
camp_raw.describe(include='object')

,Camp_Start_Date,Camp_End_Date,Category1,Category2
count,65,65,65,65
unique,58,54,3,7
top,30-Jan-07,07-Nov-07,First,C
freq,2,5,44,16


In [18]:
# lets check what those unique values are
print(camp_raw.Category1.unique().tolist())
print(camp_raw.Category2.unique().tolist())

['First', 'Second', 'Third']
['B', 'C', 'F', 'E', 'D', 'G', 'A']


In [19]:
# merge train and health camp data on Health_Camp_ID
merged_train = train_raw.merge(camp_raw, on='Health_Camp_ID').drop(['Camp_Start_Date','Camp_End_Date','Registration_Date'], axis=1)
merged_train.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3
0,489652,6578,4,0,0,0,2,Third,G,2
1,507246,6578,45,5,0,0,7,Third,G,2
2,491371,6578,0,0,0,0,0,Third,G,2
3,485995,6578,0,0,0,0,0,Third,G,2
4,511519,6578,0,0,0,0,0,Third,G,2


In [20]:
# do the same for test
merged_test = test_raw.merge(camp_raw, on='Health_Camp_ID').drop(['Camp_Start_Date','Camp_End_Date','Registration_Date'], axis=1)
merged_test.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3
0,505701,6548,1,0,0,0,2,Third,G,2
1,494067,6548,0,0,0,0,0,Third,G,2
2,499854,6548,0,0,0,0,0,Third,G,2
3,509140,6548,0,0,0,0,0,Third,G,2
4,486481,6548,0,0,0,0,0,Third,G,2


In [21]:
print(first_camp.head())
print('\n')
print(second_camp.head())
third_camp.head()

   Patient_ID  Health_Camp_ID  Donation  Health_Score  Unnamed: 4
0      506181            6560        40      0.439024         NaN
1      494977            6560        20      0.097561         NaN
2      518680            6560        10      0.048780         NaN
3      509916            6560        30      0.634146         NaN
4      488006            6560        20      0.024390         NaN


   Patient_ID  Health_Camp_ID  Health Score
0      526631            6536      0.875136
1      509122            6536      0.755700
2      498864            6536      0.673181
3      515398            6536      0.722041
4      504624            6536      0.464712


,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1
3,493167,6527,4,4
4,510954,6528,2,2


In [22]:
# merge merged_train with 3 camps info dfs on 'Patient_ID','Health_Camp_ID'
merged_train_1 = merged_train[merged_train.Category1=='First'].merge(first_camp.drop('Unnamed: 4', axis=1), 'left', on=['Patient_ID', 'Health_Camp_ID'])
merged_train_2 = merged_train[merged_train.Category1=='Second'].merge(second_camp, 'left', on=['Patient_ID', 'Health_Camp_ID'])
merged_train_3 = merged_train[merged_train.Category1=='Third'].merge(third_camp, 'left', on=['Patient_ID', 'Health_Camp_ID'])
print_shape(merged_train_1)
print_shape(merged_train_2)
print_shape(merged_train_3)

shape of the df(merged_train_1): (49892, 12)
shape of the df(merged_train_2): (15114, 11)
shape of the df(merged_train_3): (10272, 12)


In [23]:
merged_train_1.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Donation,Health_Score
0,524931,6535,0,0,0,0,0,First,E,2,NaN,NaN
1,500733,6535,0,0,0,0,0,First,E,2,NaN,NaN
2,499882,6535,0,0,0,0,0,First,E,2,NaN,NaN
3,504568,6535,3,1,0,0,3,First,E,2,10.0,0.634921
4,518865,6535,0,0,0,0,0,First,E,2,NaN,NaN


# Feature extraction

In [24]:
merged_train_1['outcome_favourable'] = 1
merged_train_1.loc[merged_train_1.Health_Score.isna(),'outcome_favourable']=0
merged_train_1.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Donation,Health_Score,outcome_favourable
0,524931,6535,0,0,0,0,0,First,E,2,NaN,NaN,0
1,500733,6535,0,0,0,0,0,First,E,2,NaN,NaN,0
2,499882,6535,0,0,0,0,0,First,E,2,NaN,NaN,0
3,504568,6535,3,1,0,0,3,First,E,2,10.0,0.634921,1
4,518865,6535,0,0,0,0,0,First,E,2,NaN,NaN,0


In [25]:
merged_train_2['outcome_favourable'] = 1
merged_train_2.loc[merged_train_2['Health Score'].isna(),'outcome_favourable']=0
merged_train_2.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Health Score,outcome_favourable
0,523729,6534,0,0,0,0,0,Second,A,2,0.402054,1
1,506153,6534,0,0,0,0,0,Second,A,2,0.402054,1
2,508986,6534,0,0,0,0,0,Second,A,2,0.532780,1
3,504679,6534,0,0,0,0,0,Second,A,2,0.819905,1
4,486432,6534,0,0,0,0,0,Second,A,2,NaN,0


In [26]:
merged_train_3['outcome_favourable'] = 1
merged_train_3.loc[merged_train_3['Number_of_stall_visited'] == 0, 'outcome_favourable'] = 0
merged_train_3.loc[merged_train_3['Number_of_stall_visited'].isna(),'outcome_favourable'] = 0
merged_train_3.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Number_of_stall_visited,Last_Stall_Visited_Number,outcome_favourable
0,489652,6578,4,0,0,0,2,Third,G,2,2.0,1.0,1
1,507246,6578,45,5,0,0,7,Third,G,2,NaN,NaN,0
2,491371,6578,0,0,0,0,0,Third,G,2,NaN,NaN,0
3,485995,6578,0,0,0,0,0,Third,G,2,3.0,3.0,1
4,511519,6578,0,0,0,0,0,Third,G,2,1.0,1.0,1


In [27]:
train_dfs = [merged_train_1, merged_train_2, merged_train_3]
common_cols = list(set.intersection(*(set(df.columns) for df in train_dfs)))
merged_with_outcome = pd.concat([df[common_cols] for df in train_dfs], ignore_index=True)
ordered_cols = list(merged_train.columns)
ordered_cols.append('outcome_favourable')
merged_with_outcome = merged_with_outcome[ordered_cols]
merged_with_outcome.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,outcome_favourable
0,524931,6535,0,0,0,0,0,First,E,2,0
1,500733,6535,0,0,0,0,0,First,E,2,0
2,499882,6535,0,0,0,0,0,First,E,2,0
3,504568,6535,3,1,0,0,3,First,E,2,1
4,518865,6535,0,0,0,0,0,First,E,2,0


In [28]:
test_raw.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,02-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0
3,497447,6551,27-Aug-06,0,0,0,0,0
4,496446,6533,19-Sep-06,0,0,0,0,0


In [29]:
patient_raw.head()

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry
2,508307,0,0,0,0,3,87,46,02-Nov-02,D,BFSI
3,512612,0,0,0,0,1,75,47,02-Nov-02,D,Education
4,521075,0,0,0,0,3,None,80,24-Nov-02,H,Others


# Final datasets

In [30]:
train_data = merged_with_outcome.merge(patient_raw, on='Patient_ID')
print_shape(train_data)
train_data.head()

shape of the df(train_data): (75278, 21)


,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,...,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,524931,6535,0,0,0,0,0,First,E,2,...,0,0,0,0,None,None,None,07-Feb-04,I,NaN
1,524931,6534,0,0,0,0,0,Second,A,2,...,0,0,0,0,None,None,None,07-Feb-04,I,NaN
2,524931,6529,0,0,0,0,0,Second,A,2,...,0,0,0,0,None,None,None,07-Feb-04,I,NaN
3,524931,6523,0,0,0,0,0,Second,D,2,...,0,0,0,0,None,None,None,07-Feb-04,I,NaN
4,524931,6528,0,0,0,0,0,Third,G,2,...,0,0,0,0,None,None,None,07-Feb-04,I,NaN


In [31]:
test_data = merged_test.merge(patient_raw, on='Patient_ID')
print_shape(test_data)
test_data.head()

shape of the df(test_data): (35249, 20)


,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,505701,6548,1,0,0,0,2,Third,G,2,0,0,0,0,0,None,44,05-Feb-03,E,NaN
1,505701,6582,1,0,0,0,2,First,F,2,0,0,0,0,0,None,44,05-Feb-03,E,NaN
2,505701,6567,1,0,0,0,2,Second,D,2,0,0,0,0,0,None,44,05-Feb-03,E,NaN
3,505701,6566,1,0,0,0,2,First,E,2,0,0,0,0,0,None,44,05-Feb-03,E,NaN
4,505701,6556,1,0,0,0,2,First,E,2,0,0,0,0,0,None,44,05-Feb-03,E,NaN


# Data pre-processing

In [32]:
# convert target to category
train_data['outcome_favourable'] = train_data['outcome_favourable'].astype('category')
train_data.dtypes

Patient_ID               int64
Health_Camp_ID           int64
Var1                     int64
Var2                     int64
Var3                     int64
Var4                     int64
Var5                     int64
Category1               object
Category2               object
Category3                int64
outcome_favourable    category
Online_Follower          int64
LinkedIn_Shared          int64
Twitter_Shared           int64
Facebook_Shared          int64
Income                  object
Education_Score         object
Age                     object
First_Interaction       object
City_Type               object
Employer_Category       object
dtype: object

In [33]:
train_data.isna().sum()

Patient_ID                0
Health_Camp_ID            0
Var1                      0
Var2                      0
Var3                      0
Var4                      0
Var5                      0
Category1                 0
Category2                 0
Category3                 0
outcome_favourable        0
Online_Follower           0
LinkedIn_Shared           0
Twitter_Shared            0
Facebook_Shared           0
Income                    0
Education_Score           0
Age                       0
First_Interaction         0
City_Type             33208
Employer_Category     60075
dtype: int64

In [34]:
# drop columns City_Type, Employer_Category since we na size is huge
# drop Patient_ID, Health_Camp_ID since they are nominal
# drop First_Interaction dtype datetime and is not important
train_data.drop(['City_Type', 'Employer_Category', 'Patient_ID', 'Health_Camp_ID', 'First_Interaction'], axis=True, inplace=True)
train_data.head()

,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,outcome_favourable,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age
0,0,0,0,0,0,First,E,2,0,0,0,0,0,None,None,None
1,0,0,0,0,0,Second,A,2,1,0,0,0,0,None,None,None
2,0,0,0,0,0,Second,A,2,0,0,0,0,0,None,None,None
3,0,0,0,0,0,Second,D,2,1,0,0,0,0,None,None,None
4,0,0,0,0,0,Third,G,2,1,0,0,0,0,None,None,None


In [35]:
# repeat the same for test data
test_data.drop(['City_Type', 'Employer_Category', 'Patient_ID', 'Health_Camp_ID', 'First_Interaction'], axis=True, inplace=True)
test_data.head()

,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age
0,1,0,0,0,2,Third,G,2,0,0,0,0,0,None,44
1,1,0,0,0,2,First,F,2,0,0,0,0,0,None,44
2,1,0,0,0,2,Second,D,2,0,0,0,0,0,None,44
3,1,0,0,0,2,First,E,2,0,0,0,0,0,None,44
4,1,0,0,0,2,First,E,2,0,0,0,0,0,None,44


In [36]:
train_data.dtypes

Var1                     int64
Var2                     int64
Var3                     int64
Var4                     int64
Var5                     int64
Category1               object
Category2               object
Category3                int64
outcome_favourable    category
Online_Follower          int64
LinkedIn_Shared          int64
Twitter_Shared           int64
Facebook_Shared          int64
Income                  object
Education_Score         object
Age                     object
dtype: object

In [37]:
# type casting for train and test datasets
cols_to_cat = ['Category1', 'Category2', 'Category3', 'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared',
              'Facebook_Shared', 'Income']
for col in cols_to_cat:
    train_data[col] = train_data[col].astype('category')
    test_data[col] = test_data[col].astype('category')

In [38]:
train_data.dtypes

Var1                     int64
Var2                     int64
Var3                     int64
Var4                     int64
Var5                     int64
Category1             category
Category2             category
Category3             category
outcome_favourable    category
Online_Follower       category
LinkedIn_Shared       category
Twitter_Shared        category
Facebook_Shared       category
Income                category
Education_Score         object
Age                     object
dtype: object

In [39]:
# check no of uniques in Education_Score
train_data.Education_Score.nunique()

228

In [41]:
# replace None with 0 since we assume none = no education
train_data.loc[train_data['Education_Score'] == 'None', 'Education_Score'] = 0
train_data.Education_Score.value_counts()

0         65345
82          468
79          423
75          413
86          390
          ...  
87.375        1
84.4          1
73.575        1
71.05         1
83.825        1
Name: Education_Score, Length: 228, dtype: int64

In [42]:
# we can now convert to Education_Score to float
train_data['Education_Score']=train_data.swifter.allow_dask_on_strings().apply(lambda x: round(float(x['Education_Score']), 2),axis=1)
train_data['Education_Score'].dtypes

Dask Apply: 100%|██████████| 24/24 [00:02<00:00, 11.15it/s]


dtype('float64')

In [44]:
# check no of uniques in Age
train_data.Age.nunique()

50

In [45]:
# replace None with 0
train_data.loc[train_data['Age'] == 'None', 'Age'] = 100

100    51612
41      1843
40      1760
42      1676
43      1628
39      1477
44      1460
45      1094
37      1033
46      1025
38       996
47       885
72       792
48       736
73       710
49       644
71       593
74       540
36       477
50       446
52       418
51       416
70       394
35       360
53       315
54       222
34       191
56       189
75       167
55       147
76       131
69       118
58       111
59        88
67        87
57        68
33        68
77        48
60        46
68        45
63        37
64        33
65        30
66        29
61        27
32        24
78        17
80        10
31         8
62         7
Name: Age, dtype: int64

In [46]:
# convert to float
train_data['Age']=train_data.swifter.allow_dask_on_strings().apply(lambda x: int(x['Age']),axis=1)
train_data['Age'].dtypes

Dask Apply: 100%|██████████| 24/24 [00:02<00:00, 10.47it/s]


dtype('int64')

In [47]:
train_data.dtypes

Var1                     int64
Var2                     int64
Var3                     int64
Var4                     int64
Var5                     int64
Category1             category
Category2             category
Category3             category
outcome_favourable    category
Online_Follower       category
LinkedIn_Shared       category
Twitter_Shared        category
Facebook_Shared       category
Income                category
Education_Score        float64
Age                      int64
dtype: object

In [48]:
age = train_data['Age'].unique().tolist()
age

[100,
 42,
 56,
 41,
 71,
 36,
 43,
 37,
 47,
 51,
 38,
 72,
 45,
 39,
 46,
 44,
 59,
 50,
 48,
 74,
 55,
 70,
 54,
 40,
 53,
 66,
 34,
 58,
 52,
 73,
 80,
 49,
 35,
 64,
 61,
 33,
 68,
 57,
 69,
 75,
 77,
 67,
 76,
 32,
 65,
 60,
 63,
 78,
 62,
 31]

In [51]:
age.sort()

In [52]:
age

[31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 100]